In [1]:
%pip install pyngrok

In [2]:
# ----------------------------------------------------------------------
# 1. SETUP AND IMPORTS
# ----------------------------------------------------------------------
# Load Dataset - This line MUST be present for Drive access in Colab!
from google.colab import drive
import joblib
import pandas as pd
from flask import Flask, request, jsonify
from pyngrok import ngrok, __version__ as ngrok_version
import os
import sys
import threading
import time
import requests # Added for the test script
import json # Added for the test script

# Mount Google Drive
drive.mount('/content/drive')

# --- SETUP: Load Model and Preprocessors on Startup ---
try:
    # ⚠️ NOTE: Use the absolute path where your files are stored in Google Drive
    base_path = '/content/drive/MyDrive/Deep CSAT Project'

    model = joblib.load(os.path.join(base_path, 'model.joblib'))
    scaler = joblib.load(os.path.join(base_path, 'scaler.joblib'))
    feature_names = joblib.load(os.path.join(base_path, 'feature_names.joblib'))
    print("Models and Scalers loaded successfully from Google Drive.")

except Exception as e:
    # Exit gracefully if critical files are missing.
    print(f"ERROR: Failed to load models from Drive: {e}", file=sys.stderr)
    sys.exit(1)

# --- FLASK APP DEFINITION ---
app = Flask(__name__)

# ----------------------------------------------------------------------
# 2. API ENDPOINT
# ----------------------------------------------------------------------

# --- Root Endpoint for Health Check and Instructions ---
@app.route('/', methods=['GET'])
def home():
    """Simple health check and API instructions."""
    return jsonify({
        'status': 'API Running',
        'message': 'Send a POST request to /predict with JSON data for CSAT prediction.',
        'ngrok_version': ngrok_version
    })


@app.route('/predict', methods=['POST'])
def predict():
    """
    Receives raw customer features via POST request, preprocesses them,
    and returns a CSAT prediction.
    """
    try:
        # 1. Get raw JSON data from the request
        raw_data = request.get_json()

        # Check if raw_data is missing or empty
        if not raw_data:
            return jsonify({'error': 'No input data provided.'}), 400

        # 2. Convert to DataFrame (API handles one customer at a time)
        input_df = pd.DataFrame([raw_data])

        # --- 3. PREPROCESSING LOGIC ---

        # a. Align columns: Create a DataFrame with all 92 expected columns.
        X_predict = input_df.reindex(columns=feature_names, fill_value=0)

        # b. Scale the data using the saved scaler
        X_scaled = scaler.transform(X_predict.values)

        # --- 4. PREDICTION ---
        prediction = model.predict(X_scaled)

        # 5. Format Output
        prediction_value = int(prediction[0])
        prediction_label = "High CSAT (1)" if prediction_value == 1 else "Low CSAT (0)"

        return jsonify({
            'prediction_label': prediction_label,
            'prediction_value': prediction_value,
            'model_used': 'XGBoost (Tuned)'
        })

    except Exception as e:
        # Return a clean error message if something fails
        print(f"Prediction error: {e}", file=sys.stderr)
        return jsonify({'error': f'Prediction failed: {str(e)}'}), 400

# ----------------------------------------------------------------------
# 3. NGROK SETUP AND SERVER RUN (Most Stable)
# ----------------------------------------------------------------------
def run_tunnel():
    """Starts the ngrok tunnel in a separate thread."""

    colab_port = 5000

    # 1. Kill any existing ngrok process and tunnel connections
    try:
        ngrok.kill()
    except Exception:
        pass

    # --- STEP 1.5: EXPLICITLY SET AUTH TOKEN ---
    # ⚠️ MANDATORY: PASTE YOUR NGROK AUTH TOKEN HERE
    NGROK_AUTH_TOKEN = "35pir11lSH5XI1eaAdtDWDuoP9V_7SYe3ZAu9kz6VdA21VoA"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    print(f"\nStarting ngrok tunnel on port {colab_port}...")

    # 2. Start the ngrok tunnel manually
    try:
        public_url = ngrok.connect(colab_port).public_url
        print(f"🎉 * Ngrok Tunnel is LIVE! Public URL: {public_url} * 🎉")
        print("To stop the server, interrupt this cell (Runtime -> Interrupt execution).")

        # --- Run the client test after confirming the public URL is available ---
        # The test is wrapped in a thread to run concurrently with the Flask server
        test_thread = threading.Thread(target=run_api_test, args=(public_url,))
        test_thread.start()

    except Exception as e:
        print(f"❌ NGROK FAILED: Please check your token and ensure it is correct. Error: {e}", file=sys.stderr)
        return

    # 3. Start the Flask app
    app.run(port=colab_port, host="0.0.0.0", debug=False, use_reloader=False)


# ----------------------------------------------------------------------
# 4. API CLIENT TEST SCRIPT (Updated with user's new values)
# ----------------------------------------------------------------------
def run_api_test(ngrok_url):
    """
    Tests the deployed Flask API with a high-risk payload.
    This runs in a separate thread after the server is confirmed up.
    """
    # Delay to ensure the server is fully bound before sending the request
    time.sleep(5)

    API_URL = f"{ngrok_url}/predict"

    # Define a high-risk customer payload (Long response time, negative sentiment)
    HIGH_RISK_PAYLOAD = {
        # CRITICAL PREDICTIVE FEATURES (Updated with user's new data)
        "Response_Time_Minutes": 5.33,

        "Sentiment_Score": -0.91,     # Very negative
        "Remark_Word_Count": 15,      # Verbose complaint

        # CATEGORICAL FEATURES
        "channel_name_Phone": 1,
        "category_Billing": 1,

        # OTHER FEATURES
        "Item_price": 5000,
        "connected_handling_time": 427,

        # Tenure feature
        "Tenure Bucket_Mid-Level": 1
    }

    print("\n--- Sending High-Risk Payload to LIVE API (Client Test) ---")
    print(f"Target URL: {API_URL}")
    print(f"Data Sent: {json.dumps(HIGH_RISK_PAYLOAD, indent=2)}")

    try:
        response = requests.post(API_URL, json=HIGH_RISK_PAYLOAD)
        response.raise_for_status()

        prediction_result = response.json()

        print("\n✅ API Prediction Successful:")
        print(json.dumps(prediction_result, indent=4))

    except requests.exceptions.RequestException as e:
        print(f"\n❌ ERROR: Failed to get prediction from API.")
        print(f"Details: {e}")
        if 'response' in locals() and hasattr(response, 'text'):
            print(f"API Response Text: {response.text}")


if __name__ == '__main__':
    # Start the ngrok tunnel and the Flask app.
    # The run_tunnel function will then start the test script in its own thread.
    tunnel_thread = threading.Thread(target=run_tunnel)
    tunnel_thread.start()

    # Wait for the tunnel thread to complete its setup
    time.sleep(5)

Mounted at /content/drive


/usr/lib/python3.12/pickle.py:1760: UserWarning: [11:04:13] WARNING: /__w/xgboost/xgboost/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


Models and Scalers loaded successfully from Google Drive.

Starting ngrok tunnel on port 5000...
🎉 * Ngrok Tunnel is LIVE! Public URL: https://fec0-34-132-98-222.ngrok-free.app * 🎉
To stop the server, interrupt this cell (Runtime -> Interrupt execution).
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
